### This workbook's scripts will take a CSV of text arrays (Approvers' Titles) and output an array of those approvers' names mapped from SEMDGroups

first, lets import libraries and our csv files

In [1]:
import pandas as pd
import numpy as np
import re
import itertools
from ast import literal_eval


In [2]:
#make dataFrame
df = pd.read_csv('../Lists/RawLists/ApproversArrays.csv')
dfGroups = pd.read_csv('../Lists/RawLists/SEMDGroups.csv')
dfPrograms = pd.read_csv('../Lists/RawLists/SEMDProgramsList.csv')
dfTitles = pd.read_csv('../Lists/RawLists/titles_reconcilliation.csv')
dfEpath = pd.DataFrame(columns=['Title', 'Office', 'Division', 'Program', 'ResponseTime', 'DueReminder', 'ReviewChain', 'TargetSite', 'TargetLName', 'TargetLtype'])

next, we will convert the job-title and corresponding names to key-value pairs in a dictionary

In [3]:
#convert Groups to a dictionary, 
#set key to Group column and set value to Person column 
groupDict = dfGroups.set_index('Group').to_dict()['Person']
titleDict = dfTitles.set_index('Gtitles').to_dict()['revGtitles']

this script will remove from the dictionary any consecutive comma groups left by blank spots in the SEMDGroups 

In [4]:
def remove_consecutive_commas(text):
    text = re.sub('( ,)+', '', text)
    return text

df['ApproverTitles'] = df['ApproverTitles'].apply(lambda x: remove_consecutive_commas(x))
df['ApproverTitles'] = df['ApproverTitles'].str.split(pat=', ')

### So far we have created a list of lists containing the approver's titles and sequence for our SEMD Programs file. We have also created a dictionary from the two columns in SEMDGroups.

### Next let's attempt to map names to the Approver Titles list items so that each list in ApproverTitles has a corresponding list of names 

In [5]:
appList = df['ApproverTitles'].tolist()

In [6]:
lvals = [[groupDict.get(_, '(Blank)') for _ in row] for row in appList]

I want to clean the titles in lval so that they are reconciled with [the list in ePath](https://usepa.sharepoint.com/sites/R5eSign/Lists/ePath/all.aspx?FilterField1=Division&FilterValue1=SEMD&FilterType1=Choice&viewid=8eea2505%2Df626%2D44df%2Dab71%2Da7bea4d8eb35)

In [7]:
titleList = [[titleDict.get(title, title) for title in row] for row in appList]

# Next we will map our document names to the list of approvers' titles and names

Source: dfPrograms['Document']
Target: df['Document']

In [8]:
#add the new columns to our main data frame
df.insert(0, 'Document', dfPrograms['Document'])

In [9]:
df.insert(1, 'Program', dfPrograms['Program'])

## Now let's create a name list that meets the specifications of the ePath system's 'ReviewerNames' column.

Each record in the ReviewerNames column has an arbitrary number of entries in the following pattern:

**Number(int)|Stage(str)|Title(str)|lName,fName[;lName,fName] (str)|allRequired(bool)|splitStage(bool)|[stageName]~\n (str) __last line no tilde__**

In [10]:
reviewString = [[f'{x+1}|S{"0" if x<10 else ""}{x+1}|{title}|{name}|false|false|~' for x, (title, name) in enumerate(zip(titleList[i],lvals[i]))] for i in range(len(titleList))] 
for s in reviewString:
    for s[-1] in s:
        s[-1] = s[-1].rstrip('~')

In [11]:
reviewStringsNew = ['\n'.join(strings) for strings in reviewString]

## **__Great Job!__**
<details>
<summary>This fundamental string will need to be refined as noted below</summary>
<br>
- remove trailing tilde on final string line of each list
- possible: add carriage return after tilde on each string
- blanks must be replaced by empty strings
- check w/ lauren about any 'all must approve' stages
- 
</details>

### the following section will assemble a dataframe into an _ePath_ compatible format (below)
Title (Document), Office(R05), Division(SEMD), Program, ResponseTime, DueReminder, ReviewChain, TargetSite, TargetLName, TargetLType

In [12]:
#format dfEpath to ePath's standards
dfEpath['Title'] = df['Document']
dfEpath['Office'] = 'R05'
dfEpath['Division'] = 'SEMD'
dfEpath['Program'] = df['Program']
dfEpath['ResponseTime'] = '7'
dfEpath['DueReminder'] = '1,1'
dfEpath['ReviewChain'] = reviewStringsNew
dfEpath['TargetSite'] = 'https://XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX'
dfEpath['TargetLName'] = 'SEMD'
dfEpath['TargetLtype'] = 'Library'



## End Scripting
### Save dataframe to .csv

In [13]:
#write the resulting dataFrame to a CSV in the draft files folder
df.to_csv('../Lists/DraftLists/titlesMapped_cleanedApproverLists.csv')
dfEpath.to_csv('../Lists/DraftLists/ePathListEntries.csv')